In [ ]:
%%writefile .gitignore
pythonpractice/
__pycache__/
*.py[cod]
*.so
.ipynb_checkpoints/
.DS_Store
.vscode/
.env
*.env
*.nbconvert.ipynb

In [ ]:
!cat .gitignore

In [ ]:
#Gordon Growth Model Stock Valuation

def get_ggm(*args):
    cost_of_equity, dps, growth_rate, current_price = args
    
    if cost_of_equity <= growth_rate:
        raise ValueError('Cost of Equity must be larger than the growth rate for the model to be valid')
    
    intrinsic_value = dps / (cost_of_equity - growth_rate)
    
    print(f"The intrinsic value is: ${intrinsic_value:.2f}")
    
    # Optional: check if current price is within ±2%
    lower_bound = intrinsic_value * 0.98
    upper_bound = intrinsic_value * 1.02
    
    if lower_bound <= current_price <= upper_bound:
        print("Stock is Fairly Valued ✅")
    elif current_price < lower_bound:
        print("Stock is Undervalued 📉")
    else:
        print("Stock is Overvalued 📈")

try:
    r = float(input('Enter the Cost of Equity : '))
    g = float(input('Enter the Growth Rate : '))
    d1 = float(input('Enter the Dividend Per Share one period from today: '))
    p = float(input('Enter the Current Stock Price: '))

    get_ggm(r, d1, g, p)

except ValueError as e:
    print(f'Error: {e}')


In [ ]:
pip install pandas numpy yfinance matplotlib seaborn plotly

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import random

In [ ]:
#Import Any Stock from Yahoo Finance using Ticker
#auto_adjust = False gives Adj Close column

mastercard = yf.Ticker('MA')
ma_data = mastercard.history(period='1y', auto_adjust = False)
ma_data.head(10)

In [ ]:
ma_data.info()

In [ ]:
#Get Daily Returns and commit column to dataframe memory

ma_data['Daily Return'] = ma_data['Adj Close'].pct_change(1) * 100
ma_data

In [ ]:
#Replace NaN with value

ma_data.replace(np.nan, 0 , inplace = True)
ma_data

In [ ]:
#Cleaning data

ma_data = ma_data.reset_index()
ma_data

In [ ]:
#Cleaning data

ma_data = ma_data.drop(columns=['level_0', 'index'])
ma_data

In [ ]:
ma_data.describe().round(2)

In [ ]:
fig = px.line(title = 'Mastercard (MA) Adjusted Closing Price')
fig.add_scatter(x = ma_data['Date'], y = ma_data['Adj Close'], name = 'Adj Close')

In [ ]:
#Create function to be used for plotting data throughout project

def plot_data (df, title):
    fig = px.line(title = title)

    for i in df.columns[1:]:
        fig.add_scatter(x = df['Date'], y = df[i], name = i)
        fig.update_traces(line_width = 5)
        fig.update_layout({'plot_bgcolor': "white"})
    fig.show()

In [ ]:
plot_data(ma_data.drop(['Volume', 'Daily Return', 'Stock Splits', 'Dividends'], axis =1), 'Mastercard (MA) Stock Price')

In [ ]:
plot_data(ma_data.iloc[:,[0,6]], 'Mastercard (MA) Stock Daily Volume')

In [ ]:
plot_data(ma_data.iloc[:,[0,9]], 'Mastercard (MA) Daily Return')

In [ ]:
#Pie Chart of Std Dev Moves using pd.cut

ma_returns = ma_data['Daily Return']
std = ma_returns.std()

bins = [0, std, 2*std, np.inf]
labels = ['< 1σ Move', '1-2σ Move', '≥ 2σ Move']

ma_data['Return Category'] = pd.cut(ma_returns.abs(), bins=bins, labels=labels, include_lowest=True)

category_counts = ma_data['Return Category'].value_counts().sort_index()

plt.figure(figsize=(6,6))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%',
        startangle=140, wedgeprops={'edgecolor': 'black'})
plt.title('Distribution of Daily Returns by Standard Deviation Moves')
plt.show()

In [ ]:
# Build candlestick chart

fig = go.Figure(data=[
    go.Candlestick(
        x=ma_data.index,
        open=ma_data['Open'],
        high=ma_data['High'],
        low=ma_data['Low'],
        close=ma_data['Close'],
        name='Candlestick'
    )
])
ma_data['SMA14'] = ma_data['Close'].rolling(window=14).mean()
ma_data['SMA21'] = ma_data['Close'].rolling(window=21).mean()

for period, color in zip([14, 21], ['blue', 'orange']):
    fig.add_trace(go.Scatter(
        x=ma_data.index,
        y=ma_data[f'SMA{period}'],
        mode='lines',
        line=dict(color=color,width=2),
        name=f'{period}-Day SMA'
    ))
    

fig.update_layout(
    title='Mastercard Candlestick Chart With SMA',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    xaxis_rangeslider_visible=False
)

fig.show()


In [ ]:
#Bring in portfolio of stocks from Yahoo Finance using Tickers

tickers = ['BA', 'WMT', 'QCOM', 'XOM', 'LULU', 'MA']
stocks = yf.download(tickers, start='2019-01-01', end='2024-12-31', auto_adjust = False)['Close']
print(stocks.head())

In [ ]:
#Verify there are no NaN values

stocks.isnull().sum()

In [ ]:
stocks

In [ ]:
#Clean Data to be used for analysis

stocks.columns.name = None
stocks

In [ ]:
#Clean Data to be used for analysis

stocks.index = pd.to_datetime(stocks.index)
stocks = stocks.reset_index()
stocks.insert(0, 'Index', range(1, len(stocks) +1))
stocks

In [ ]:
#Clean Data to be used for analysis

stocks.drop('Index', axis=1, inplace=True)
stocks

In [ ]:
#Get a Daily Returns dataframe for analysis

daily_returns_df = stocks.iloc[:, 1:].pct_change() * 100
daily_returns_df

In [ ]:
daily_returns_df.replace(np.nan, 0, inplace = True)
daily_returns_df

In [ ]:
daily_returns_df.insert(0, 'Date', stocks['Date'])
daily_returns_df

In [ ]:
plot_data(stocks, 'Portfolio Adjusted Closing Prices')

In [ ]:
plot_data(daily_returns_df, 'Portfolio Daily Returns')

In [ ]:
fig = px.histogram(daily_returns_df.drop(columns = ['Date']))
fig.update_layout({'plot_bgcolor': "white"})

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(daily_returns_df.drop(columns = ['Date']).corr(), annot = True, cmap = 'crest');

In [ ]:
sns.pairplot(daily_returns_df);

In [ ]:
#Function to scale prices of all stocks in portfolio to begin at same point

def price_scale(raw_prices):
    scaled_prices = raw_prices.copy()
    for i in raw_prices.columns[1:]:
        scaled_prices[i] = raw_prices[i]/raw_prices[i][0]
    return scaled_prices

In [ ]:
price_scale(stocks)

In [ ]:
plot_data(price_scale(stocks), 'Portfolio Adjusted Close Data Scaled')

In [ ]:
#Function to obtain random weights to be used in Monte Carlo Simulation

def portfolio_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())

    weights = weights/np.sum(weights)
    return weights

In [ ]:
weights = portfolio_weights(6)
print(weights)

In [ ]:
weights.sum()

In [ ]:
portfolio_df = stocks.copy()
scaled_df = price_scale(portfolio_df)
scaled_df

In [ ]:
investment = 10000000
for i, stock in enumerate(scaled_df.columns[1:]):
    portfolio_df[stock] = weights[i] * scaled_df[stock] * investment
portfolio_df.round(1)

In [ ]:
#Function to be used in Monte Carlo Simulation to determine optimal asset allocation

def asset_allocation(df, weights, investment):
    portfolio_df = df.copy()
    scaled_df = price_scale(df)

    for i, stock in enumerate(scaled_df.columns[1:]):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * investment

    portfolio_df['Portfolio Value'] = portfolio_df[portfolio_df != 'Date'].sum(axis = 1, numeric_only = True)

    portfolio_df['Portfolio Daily % Return'] = portfolio_df['Portfolio Value'].pct_change(1) * 100
    portfolio_df.replace(np.nan, 0, inplace = True)

    return portfolio_df

In [ ]:
n = len(stocks.columns)-1
print(f'Number of Stocks In The Portfolio: {n}')
weights = portfolio_weights(n).round(6)
print(f'Portfolio weights: {weights}')

portfolio_df = asset_allocation(stocks, weights, 10000000)
portfolio_df.round(2)

In [ ]:
plot_data(portfolio_df[['Date', 'Portfolio Daily % Return']], 'Portfolio Daily % Return')

plot_data(portfolio_df.drop(['Portfolio Value', 'Portfolio Daily % Return'], axis = 1), 'Individual Stock Position Values')

plot_data(portfolio_df[['Date', 'Portfolio Value']], 'Total Portfolio Value')

In [ ]:
std = portfolio_df['Portfolio Daily % Return'].std()

bins = [0, std, 2*std, np.inf]
labels = ['< 1σ Move', '1-2σ Move', '≥ 2σ Move']

portfolio_df['Return Category'] = pd.cut(portfolio_df['Portfolio Daily % Return'].abs(), bins=bins, labels=labels, include_lowest=True)

category_counts = portfolio_df['Return Category'].value_counts().sort_index()

plt.figure(figsize=(6,6))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%',
        startangle=140, wedgeprops={'edgecolor': 'black'})
plt.title('Portfolio Daily Returns by Standard Deviation Moves')
plt.show()

In [ ]:
#Function to be used for Monte Carlo Simulation

def sim_engine(weights, investment):
    portfolio_df = asset_allocation(stocks, weights, investment)

    roi = ((portfolio_df['Portfolio Value'][-1:] -
            portfolio_df['Portfolio Value'][0]) /
            portfolio_df['Portfolio Value'][0]) * 100

    portfolio_daily_return = portfolio_df.drop(columns = ['Date', 'Portfolio Value', 'Portfolio Daily % Return'])
    portfolio_daily_return = portfolio_daily_return.pct_change(1)

    expected_return = np.sum(weights * portfolio_daily_return.mean()) * 252
    covariance = portfolio_daily_return.cov() * 252
    expected_vol = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    #10YR US Treasury rate at the time of analysis - taken from ycharts

    rf = 0.0427

    sharpe_ratio = (expected_return - rf) / expected_vol
    return expected_return, expected_vol, sharpe_ratio, portfolio_df['Portfolio Value'][-1:].values[0], roi.values[0]

In [ ]:
investment = 10000000
portfolio_metrics = sim_engine(weights, investment)

print(f'Expected Portfolio Annual Return = {portfolio_metrics[0] *100:.2f}%')
print(f'Portfolio Std Dev (Vol) = {portfolio_metrics[1] *100:.2f}%')
print(f'Sharpe Ratio = {portfolio_metrics[2]:.2f}')
print(f'Portfolio Value = ${portfolio_metrics[3]:.2f}')
print(f'ROI = {portfolio_metrics[4]:.2f}%')

In [ ]:
#Monte Carlo Simulation

runs = 1000
investment = 10000000

weights_runs = np.zeros((runs, n))
sharpe_ratio_runs = np.zeros(runs)
expected_return_runs = np.zeros(runs)
vol_runs = np.zeros(runs)
roi_runs = np.zeros(runs)
portfolio_value_runs = np.zeros(runs)

for i in range(runs):
    weights = portfolio_weights(n)
    weights_runs[i,:] = weights

    expected_return_runs[i], vol_runs[i], sharpe_ratio_runs[i], portfolio_value_runs[i], roi_runs[i] = sim_engine(weights, investment)
    print(f'Simulation Run = {i}')
    print(f'Weights : {weights_runs[i].round(3)}, Portfolio Value : ${portfolio_value_runs[i].round(2)}, Sharpe Ratio : {sharpe_ratio_runs[i].round(2)}')
    print('\n')

In [ ]:
#View max sharpe ratio found in n Monte Carlo Simulations

sharpe_ratio_runs.max()

In [ ]:
#View position of max sharpe ratio found in n Monte Carlo Simulations

sharpe_ratio_runs.argmax()

In [ ]:
#View optimal weights of portfolio found in n Monte Carlo Simulations

weights_runs[sharpe_ratio_runs.argmax(), :]

In [ ]:
#View optimal portfolio metrics

optimal_portfolio_return, optimal_vol, optimal_sharpe_ratio, optimal_portfolio_value, optimal_roi =sim_engine(weights_runs[sharpe_ratio_runs.argmax(), :], investment)

In [ ]:
print(f'Optimal Portfolio Metrics based on {runs} Monte Carlo Sim Runs:')
print(f'   -Expected Annual Return : {optimal_portfolio_return * 100:.02f}%')
print(f'   -Std Dev (Vol) : {optimal_vol * 100:.02f}%')
print(f'   -Sharpe Ratio : {optimal_sharpe_ratio:.02f}')
print(f'   -Dollar Value : ${optimal_portfolio_value:,.02f}')
print(f'   -ROI : {optimal_roi:.02f}%')

In [ ]:
#Create DataFrame of corresponding Vol, Return, and Sharpe Ratio to be used for Markowitz Efficient Frontier Visual

sim_result_df = pd.DataFrame({'Volatility': vol_runs.tolist(), 'Portfolio Return': expected_return_runs.tolist(), 'Sharpe Ratio': sharpe_ratio_runs.tolist()})
sim_result_df

In [ ]:
#Markowitz Efficient Frontier Visual

fig = px.scatter(sim_result_df, x = 'Volatility', y = 'Portfolio Return', color = 'Sharpe Ratio', size = 'Sharpe Ratio', hover_data = ['Sharpe Ratio'])
fig.update_layout({'plot_bgcolor':"white"})
fig.show()

In [ ]:
#Create Visual of Optimal Portfolio

fig = px.scatter(sim_result_df, x = 'Volatility', y = 'Portfolio Return', color = 'Sharpe Ratio', size = 'Sharpe Ratio', hover_data = ['Sharpe Ratio'])
fig.add_trace(go.Scatter(x = [optimal_vol], y = [optimal_portfolio_return], mode = 'markers', name = 'Optimal Point', marker = dict(size=[40], color = 'green')))
fig.update_layout(coloraxis_colorbar = dict(y= 0.7, dtick = 5))
fig.update_layout({'plot_bgcolor':"white"})
fig.show()